### 1️⃣ 데이터 준비 및 전처리

#### 카테고리 선택 및 라벨 재정렬
```python
def filter_categories(dataset, categories):
    target_names = dataset.target_names
    selected_idx = [target_names.index(c) for c in categories]
    data_filtered, target_filtered = [], []
    for text, label in zip(dataset.data, dataset.target):
        if label in selected_idx:
            new_label = selected_idx.index(label)  # 라벨 재정렬
            data_filtered.append(text)
            target_filtered.append(new_label)
    return data_filtered, target_filtered, categories

train_data, train_target, target_names = filter_categories(newsgroups_train, categories)
test_data, test_target, _ = filter_categories(newsgroups_test, categories)
```

#### 텍스트 정제 (헤더, 풋터, 인용문 제거)
```python
import re
def clean_text(text):
    text = re.sub(r'^From:.*\n', '', text, flags=re.MULTILINE)
    text = re.sub(r'^Subject:.*\n', '', text, flags=re.MULTILINE)
    text = re.sub(r'\n--\n.*$', '', text, flags=re.DOTALL)
    text = re.sub(r'(^|\n)[>|:].*', '', text)
    return text

train_data = [clean_text(t) for t in train_data]
test_data = [clean_text(t) for t in test_data]

len(train_data), len(train_target), len(test_data), len(test_target)
```

### 2️⃣ 벡터화 및 기초 모델
```python
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
```

#### BOW 벡터화
```python
cv = CountVectorizer(max_features=2000, min_df=5, max_df=0.5)
x_train_cv = cv.fit_transform(train_data)
x_test_cv = cv.transform(test_data)
```
#### Multinomial Naive Bayes 학습
```python
nb = MultinomialNB()
nb.fit(x_train_cv, train_target)
print("BOW + MNB:", nb.score(x_train_cv, train_target), nb.score(x_test_cv, test_target))
```
#### TF-IDF 벡터화
```python
tfidf = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.5)
x_train_tfidf = tfidf.fit_transform(train_data)
x_test_tfidf = tfidf.transform(test_data)
```
#### TF-IDF + MNB
```python
nb_tfidf = MultinomialNB()
nb_tfidf.fit(x_train_tfidf, train_target)
print("TF-IDF + MNB:", nb_tfidf.score(x_train_tfidf, train_target), nb_tfidf.score(x_test_tfidf, test_target))
```

### 3️⃣ Logistic Regression 및 규제
```python
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(
    x_train_tfidf, train_target, test_size=0.2, stratify=train_target, random_state=42
)
```
#### 일반 Logistic Regression
```python
lr = LogisticRegression(max_iter=1000)
lr.fit(x_train_tfidf, train_target)
print("LogisticRegression:", lr.score(x_train_tfidf, train_target), lr.score(x_test_tfidf, test_target))
```
#### RidgeClassifier (L2 규제)
```python
rc = RidgeClassifier(alpha=15)
rc.fit(x_train, y_train)
print("RidgeClassifier:", rc.score(x_train, y_train), rc.score(x_val, y_val))
```
#### L1 규제 Logistic (특성 선택)
```python
l1_lr = LogisticRegression(penalty='l1', max_iter=1000, solver='l
```

### 분류 모델 요약

#### 1. RidgeClassifier
- **설명**: 선형 모델에 L2 규제를 적용한 분류기. 과적합을 완화하고 가중치를 안정적으로 학습.
- **특징**:
  - 희소(sparse) 데이터와 잘 맞음
  - 학습 속도가 빠름
  - 규제(alpha)로 과적합 조절 가능

---

#### 2. DecisionTreeClassifier
- **설명**: 데이터를 기준(feature)으로 나누어 트리 구조로 분류하는 모델.
- **특징**:
  - 직관적인 규칙 기반 분류
  - 과적합 위험이 있음 (특히 깊은 트리)
  - 시각화가 쉬움

---

#### 3. RandomForestClassifier
- **설명**: 여러 개의 Decision Tree를 만들어 투표 방식으로 예측하는 앙상블 모델.
- **특징**:
  - 과적합 방지 및 일반화 성능 향상
  - 변수 중요도(feature importance) 제공
  - 안정적이고 강력한 성능

---

#### 4. GradientBoostingClassifier
- **설명**: 이전 트리의 오차를 보정하면서 순차적으로 트리를 학습하는 부스팅 기반 앙상블.
- **특징**:
  - 강력한 예측 성능
  - 학습 속도 느릴 수 있음
  - 과적합 방지를 위해 learning_rate, n_estimators 조절 필요